# LangChain Retrival
- Source --> Load --> Transform (slice context) --> Embed (text to vector) --> Store (in Vector DB) --> Retrieve (from DB)

# Document Loader
- 最終目的: 存到 Vector DB
- **要注意被 LangChain 封裝過的第三方套件**
  - eg: `pypdf` 原生與 LangChain 包裝過的版本，參數會有差
  - 建議用原生套件，再包成 LangChain 可用
## Lab 1: dl_pdf
- Document Loader (可以是 Word，PDF，S3，網站等等的文檔來源)
- ref: https://python.langchain.com/docs/integrations/document_loaders/

In [ ]:
# poetry add pypdf
# 由 LangChain 包裝的 pypdf
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./qa.pdf")

docs = loader.load()

print(docs[0].page_content)

# for doc in docs:
#     print(doc.page_content)

## Lab 2: dl_web
- LangChain 整合 BS4 做網頁爬蟲

In [ ]:
# poetry add beautifulsoup4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.drmaster.com.tw/bookinfo.asp?BookID=MP22437")

docs = loader.load()

print(docs[0].page_content)

## Lab 3: dl_word

In [ ]:
# poetry add docx2txt
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./第三周直播流程.docx")

docs = loader.load()

print(docs[0].page_content)

# Text Split
## Lab 4: Text Split
- 把大文本切成更小的片段 (讓向量搜尋時，可以搜尋到最相似語意的片段)
  - 要設定部分重疊
  - **OpenAI 官方預設** 800 : 400 (Chunk Size : Chunk Overlap)
- 建議一律使用 Recursive 的方式
- Concept: https://chunkviz.up.railway.app/

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = PyPDFLoader("./qa.pdf")

docs = loader.load()

# splitter = RecursiveCharacterTextSplitter() # recommanded

# chunks = splitter.split_documents(docs)

# for chunk in chunks:
#     print(chunk)
#     print("===")

# ---

# set chunk size and chunk overlap
# overlap 由 LangChain 演算法決定，不一定必定會有
splitter = RecursiveCharacterTextSplitter(    
    chunk_size=200,
    chunk_overlap=30)

chunks = splitter.split_documents(docs)

for chunk in chunks:
    print(chunk)
    print("===")
    

# Embedding
## Lab 6: embedding
- Text to Vector

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com/openai/deployments/text-embedding-small/embeddings?api-version=2023-05-15", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

# Store
## Lab 7: store
- Store vector into Vector DB

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

loader = PyPDFLoader("./qa.pdf")

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

# from_documents 是給 LangChain 的 documnet loader 的 chunk 使用
# from_list 是給一般的 list 使用
# review: http://192.168.70.128:6333/dashboard#/collections
qdrant = Qdrant.from_documents(
    chunks,
    embeddings_model,
    url="localhost",
    collection_name="subsidy_qa",
    force_recreate=True,
)

# Retrieval
## Lab 8: retrieval
- Maximum marginal relevance retrieval：更多元觀點的搜尋方式
  - 智能客服往往不需要多元觀點，而是客戶需要準確答案

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

loader = PyPDFLoader("./qa.pdf")

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

qdrant = Qdrant.from_documents(
    chunks,
    embeddings_model,
    url="localhost", 
    collection_name="subsidy_qa",
    force_recreate=True,
)

# Method 1
# as_retriever return `Runnalbe` inteface in LangChain
# therefore, the `.invoke()` method to execute a `Runnable`
retriever = qdrant.as_retriever(search_kwargs={"k": 3}) # k : 3 搜尋三個最接近的文本; openAI k : 20
docs = retriever.invoke("托育補助的發放金額如何計算？")
# docs = retriever.get_relevant_documents("托育補助的發放金額如何計算？") # old version
print(len(docs))
print(docs)

# Method 2 - 多元觀點
# retriever = qdrant.as_retriever(search_type="mmr") # "mmr": 找有相似卻又不那麼相似 (對立、多元意見)
# docs = retriever.invoke("托育補助的發放金額如何計算？")
# print(docs)

# Method 3
# result = qdrant.similarity_search_with_score("托育補助的發放金額如何計算？", k=3)
# print(result)

# RAG (Retrieval Augmented Generation)
- Workflow
  1. Source (eg. PDF) => Embedding Model (Text2Vector) => Vector DB (Already)
  2. User **Query** => Embedding Model => Fetch relevant **Context** in Vector DB according to the similarity
  3. Query + Context => LLM => Response
- Keypoints
  - Embedding Model with specific language
  - LLM Smart
- RAG Advances
  - 提高準確性與真實性，有效減少幻覺
  - 增強解釋性
  - 資源的節省有效利用
  - 靈活與可擴展性，接入企業資料更有彈性
  - 安全性高，可以減少不必要資訊送入 AI model
## Lab 9: RAG

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.vectorstores import Qdrant
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

loader = PyPDFLoader("./qa.pdf")

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

qdrant = Qdrant.from_documents(
    chunks,
    embeddings_model,
    url="localhost", 
    collection_name="subsidy_qa",
    force_recreate=True,
)

# LangChain RAG 要先轉乘 retriever
retriever = qdrant.as_retriever(search_kwargs={"k": 3})

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

# <context> </context> 是 LangChain prompt template 保留字，用於做文本切割分隔 (亦為 Prompt 技巧)
prompt = ChatPromptTemplate.from_template("""請回答依照 context 裡的資訊來回答問題，超過範圍的請不要回答:
<context>
{context}
</context>
Question: {input}""")

# chain one (document chain)
document_chain = create_stuff_documents_chain(model, prompt)
# chain two (retrieval chain)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "請問我二寶是身障，可以補助多少錢？"})
print(response["answer"])

# Advance RAG
1. Embedding Model + **Sparse Vector** (關鍵字) when inputing RAG Data
2. **Hybrid Search** (Dense + Sparse Vector) / **Re-rank Model** when fetching relevant context
3. **Query** 換句話說多次 (不同文句表達相同意義)
4. **Validator** (Guard) / **Ensemble** (整合多結果)

# LangChain Agent
- 它是由 LLM 和 prompt 所結合的 chain，由 Agent 負責決定根據給定情況下一步應採取什麼動作
- 它接收的輸入有下面幾種：
    1. 可用的工具列表
    2. 使用者輸入
    3. 先前執行的步驟（Intermediate Steps）
- 根據這些輸入，Agent 會返回下一個要採取的動作或給使用者的最終回應。這些回應被稱為 AgentAction 或 AgentFinish
- LangChain 裡的 Agent 功能不會被放棄
  - 官方鼓勵使用新的 **LangGraph**

# Tools
- Tool call
- Tools 是 Agent 調用以完成特定任務的 function
- 在設計這些工具時，有兩個重要的考慮因素：
    1. 給 Agent 提供正確的工具：這代表著工具要對應目標任務，有特定的用途和功能
    2. 以最有用的方式描述這些工具：工具的描述應清晰，以便 Agent 能夠更有效地使用它們
## Toolkits
- Langchain 提供了稱為 Toolkits 的內建工具組
  - 例如 Langchain 就內建了維基百科、數學計算等工具
- 這些工具組提供了多樣的功能，以供開發者快速入門，並使用戶能夠更容易地建立客製化的解決方案
## Zero-shot ReAct
- 這種 Agent 使用 ReAct 框架
- 根據工具的描述來決定使用哪一個工具
- 這個 Agent 要求為每個工具提供一個描述，所以描述要好好寫
- 這是算是最通用的 Agent
- ReAct agent 如果沒有特別指稱的話，ReactAgent 就是 zero-shot
- 可以理解為一個策略，引導 LLM Step by Step: 思考 -> 行動 -> 思考 -> 行動 ...
## Custom Tools
- 有兩種方式 Custom
- 再加碼 Tool Call 技巧的串接

- Notes
  - vs Reasoning Model

## Lab 10: toolkit

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import AzureChatOpenAI


chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是熱心助人的 AI，同時也是地理高手。要負責回答地理想關的問題。"),
    ("human", "你好！"),
    ("ai", "你好~"),
    ("human", "{user_input}"),
])

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

# math chain
llm_math = LLMMathChain.from_llm(model)
# llm chaim
llm_chain = chat_template | model

math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="你是計算數學問題的工具，請依據使用者所提供的資訊，告訴我計算出來的結果"
)  

geo_tool = Tool(
    name="Geography master",
    func=llm_chain.invoke,
    description="你是地理問題的好用工具，請依據使用者所提供的資訊，告訴我國家的首都。"
)

tools = [math_tool, geo_tool]

# 注意 ReAct Prompt !!!
prompt = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "請問一個三邊為 3 4 5 的三角形的面積是多少？"})
print(response)


response = agent_executor.invoke({"input": "請告訴我新加坡的首都是哪裡？"})
print(response)

response = agent_executor.invoke({"input": "請告訴我南非的首都是哪裡？"})
print(response)

## Lab 11: custom tool
- Most usefull

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import Annotated, Type
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import AzureChatOpenAI
from langchain.tools import BaseTool
from typing import Optional, Union
import requests

@tool
def square_area(edge) -> float:
    """計算正方形的面積時，使用這個工具。"""
    return float(edge)**2

triangle_desc = (
    "use this tool when you need to calculate the area of a triangle"
    "To use the tool, you must provide all of the following parameters "
    "{'a_side', 'b_side', 'c_side'}."
)

# 基於 Class 的工具比較好用
class TriangleTool(BaseTool):
    name: str = "Triangle area calculator"
    description: str = triangle_desc
    
    # 主邏輯
    def _run(
        self,
        sides: Optional[Union[int, float, str]] = None,
    ):
        import ast
        parsed_dict = ast.literal_eval(sides)

        a_side = parsed_dict['a_side']
        b_side = parsed_dict['b_side']
        c_side = parsed_dict['c_side']

        if not all([a_side, b_side, c_side]):
            print(a_side, b_side, c_side)
            raise ValueError("You must provide all three sides of the triangle")
        if a_side <= 0 or b_side <= 0 or c_side <= 0:
            raise ValueError("All sides must be greater than 0")
        if a_side + b_side <= c_side or a_side + c_side <= b_side or b_side + c_side <= a_side:
            raise ValueError("The sum of any two sides must be greater than the third side")
        # calculate the area (海龍公式)
        s = (a_side + b_side + c_side) / 2
        area = (s * (s - a_side) * (s - b_side) * (s - c_side)) ** 0.5
        return area

md5_desc = (
    "This tool parses input string for md5."
    "To use the tool, you must find what is the input string."
)


class MD5EncryptionInput(BaseModel):
    """Input for MD5Tool."""
    text: Annotated[str, Field(description="要加密成MD5的文字")]

def md5_request(text: str):
    response = requests.get("https://api.asc-x.com/md5.php", params={"text": text})
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        raise Exception("Error calling MD5 encryption API")

class MD5Tool(BaseTool):
    name: str = "MD5 Tool"
    description: str = md5_desc
    args_schema: Type[BaseModel] = MD5EncryptionInput
    
    # 主邏輯
    def _run(
        self,
        text: Optional[str] = None,
    ):
        
        return md5_request(text)


tools = [square_area, TriangleTool(), MD5Tool()]

prompt = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

                                      
Question: {input}
Thought:{agent_scratchpad}
""")

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "請問一個三邊為 3, 4, 5 的三角形，面積是多少？"})
print(response)

response = agent_executor.invoke({"input": "請問邊 10 的正方形的面積是多少？"})
print(response)

response = agent_executor.invoke({"input": "請問「我愛5566」的 MD5 加密是什麼？"})
print(response)

# LangGraph 的 Agent
- DAG (有向無環圖)

## 測試 AI Agent 的三步驟
1. 收斂關鍵場景: 與業務部門列出 100–200 條「最怕答錯」 的情境（合規條文、價格算法、治療指南…）。
2. 生成＋人工校對: 利用文件透過 AI 來 做一個 Q&A 生成器，再由客戶的專家花大量時間確認答案。
3. 持續增補: 將線上失敗案例自動寫回題庫，達到「用真實錯誤反餵模型」的循環。

In [ ]:
# poetry add langgraph
from typing import List, Dict, Any
from typing_extensions import TypedDict
from langchain_core.tools import BaseTool
import requests
from pydantic import BaseModel, Field
from typing import Annotated, Type
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt.chat_agent_executor import create_react_agent
from langgraph.graph import StateGraph, END

class MD5EncryptionInput(BaseModel):
    """Input for MD5Tool."""
    text: Annotated[str, Field(description="要加密成MD5的文字")]

class MD5Tool(BaseTool):
    name : str = "md5_encrypt"
    description : str = "把文字加密成 MD5"
    args_schema : Type[BaseModel] = MD5EncryptionInput

    def _run(self, text: str):
        r = requests.get("https://api.asc-x.com/md5.php", params={"text": text})
        r.raise_for_status()
        return r.json()

tools = [ MD5Tool()]

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

react_agent = create_react_agent(
    model=model,
    tools=tools,
    prompt="你是一個很棒的小幫手，要使用tools來幫助使用者解決問題。",
) 

class AgentState(TypedDict):
    messages: List[Dict[str, Any]]  

g = StateGraph(AgentState)
g.add_node("agent", react_agent)  # 單節點
g.set_entry_point("agent")
g.add_edge("agent", END)          # 一回合就結束

workflow = g.compile()            # LangGraph Runnable

print(workflow.invoke({"messages": [{"role": "user", "content": "「我愛5566」的 MD5？"}]}))

# 第四周作業
1. 寫一隻 Python 程式「area. Py」，使用 LangChain 的 Agent，寫兩個 Custom tool，一個是可以計算圓形面積，一個是可以計算三角形面積。
2. 使用兩個 Prompt ，呼叫該 Agent ，提供計算用的數值，以計算圓形和三角形面積。

- 評分標準：
1. 程式碼 Custom tool 的邏輯正確。（兩個 tool 分別各 30 分）
2. 程式碼可以順利跑起來。（20 分）
3. 面積計算結果與 prompt 相符。（兩個分別各 10 分）

- 上傳作業說明：
1. 截三部份圖片，第一部份為 area. Py 之程式碼，第二部份為圓形面積計算的 prompt 與結果，第三部分為三角形面積計算的 prompt 與結果。
2. 截圖必需要能看清楚與完整程式碼與結果，並且檔名要說明該圖片是什麼內容。
3. 壓縮後的資料夾檔名請統一命名：學號_作業名稱，zip 需學員座號作為辨識是否有繳交的評分結果

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import Annotated, Type
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import AzureChatOpenAI
from langchain.tools import BaseTool
from typing import Optional, Union

triangle_desc = (
    "use this tool when you need to calculate the area of a triangle"
    "To use the tool, you must provide all of the following parameters "
    "{'a_side', 'b_side', 'c_side'}."
)

class TriangleTool(BaseTool):
    name: str = "Triangle area calculator"
    description: str = triangle_desc
    
    # 主邏輯
    def _run(
        self,
        sides: Optional[Union[int, float, str]] = None,
    ):
        print(sides)
        import ast
        parsed_dict = ast.literal_eval(sides)

        a_side = parsed_dict['a_side']
        b_side = parsed_dict['b_side']
        c_side = parsed_dict['c_side']

        if not all([a_side, b_side, c_side]):
            raise ValueError("You must provide all three sides of the triangle")
        if a_side <= 0 or b_side <= 0 or c_side <= 0:
            raise ValueError("All sides must be greater than 0")
        if a_side + b_side <= c_side or a_side + c_side <= b_side or b_side + c_side <= a_side:
            raise ValueError("The sum of any two sides must be greater than the third side")
        # calculate the area (海龍公式)
        s = (a_side + b_side + c_side) / 2
        area = (s * (s - a_side) * (s - b_side) * (s - c_side)) ** 0.5
        return area

circle_desc = (
     "use this tool when you need to calculate the area of a circle"
    "To use the tool, you must provide all of the following parameters "
    "{'radius'}."
)

class CircleTool(BaseTool):
    name: str = "Circle area calculator"
    description: str = circle_desc
    
    # 主邏輯
    def _run(
        self,
        radius: Optional[str] = None,
    ):
        if not radius:
            raise ValueError("You must provide the radius of the circle")
        import ast
        parsed_radius = ast.literal_eval(radius) # where radius might be "{'radius': 12}"
        r = parsed_radius.get("radius")
        if r <= 0:
            raise ValueError("The radius you provided is not acceptable")

        from math import pi
        area = pi * r ** 2
        return area


tools = [TriangleTool(), CircleTool()]

prompt = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

                                      
Question: {input}
Thought:{agent_scratchpad}
""")

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "請問一個三邊為 3, 4, 5 的三角形，面積是多少？"})
print(response)

response = agent_executor.invoke({"input": "請問半徑 10 的圓形面積是多少？"})
print(response)



> Entering new AgentExecutor chain...
Thought: 這是一個三邊長已知的三角形，可以用三角形面積計算器計算其面積。
Action: Triangle area calculator
Action Input: {'a_side': 3, 'b_side': 4, 'c_side': 5}{'a_side': 3, 'b_side': 4, 'c_side': 5}
6.0Thought: 我已經知道三邊為 3, 4, 5 的三角形面積是 6.0。
Final Answer: 這個三角形的面積是 6.0。

> Finished chain.
{'input': '請問一個三邊為 3, 4, 5 的三角形，面積是多少？', 'output': '這個三角形的面積是 6.0。'}


> Entering new AgentExecutor chain...
Thought: 這是一個關於圓形面積的問題，我需要使用圓形面積計算器，並提供半徑 10。
Action: Circle area calculator
Action Input: {'radius': 10}314.1592653589793Thought: 我已經得到了圓形面積的計算結果。
Final Answer: 半徑 10 的圓形面積是 314.16（四捨五入到小數點後兩位）。

> Finished chain.
{'input': '請問半徑 10 的圓形面積是多少？', 'output': '半徑 10 的圓形面積是 314.16（四捨五入到小數點後兩位）。'}
